In [52]:
import numpy as np
import pandas as pd
import Utils as ut
import json
import gzip
import ast

In [39]:
games_df = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json", lines=True)

In [46]:
games_df.sample(10)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
84690,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
84649,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
29213,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
17013,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
57912,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
11801,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
20072,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
49194,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
41805,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
103244,Home Net Games,"[Action, Adventure, Free to Play, Indie, Simul...",The Pirate: Plague of the Dead,The Pirate: Plague of the Dead,http://store.steampowered.com/app/723790/The_P...,2017-10-13,"[Free to Play, Action, Simulation, Pirates, St...",http://steamcommunity.com/app/723790/reviews/?...,"[Single-player, In-App Purchases, Steam Cloud]",Free To Play,0.0,723790.0,Home Net Games


In [51]:
games_df.tags.dtype

dtype('O')

In [50]:
ut.data_review(games_df)

TypeError: unhashable type: 'list'

<hr>

In [ ]:
reviews_df = pd.read_json(r"../Datasets/user_reviews.json/australian_user_reviews.json", lines=True)

In [35]:
# list items
reviews = []
# open file json
with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
# create data frame with data
df_user_items = pd.DataFrame(reviews)
# display df
df_user_items

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


<hr>

In [ ]:
users_df = pd.read_json(r"../Datasets/users_items.json/australian_users_items.json", lines=True)

In [36]:
# list items
reviews = []
# open file json
with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
# create data frame with data
df_user_items = pd.DataFrame(reviews)
# display df
df_user_items##88310 rows 5 columns #4 min

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


<hr>

In [ ]:
games_df.sample(10)#tiene muchos nulls